# Data Loader

This file aims to add utils functions and classes to load and handle data of the project.

### Class to import in other files to load datasets of the project

In [1]:
import os
import pandas as pd

class DataLoader:
    def __init__(self, data_folder):
        self.data_folder = data_folder
        self.files = {
            "reviews": "reviews.csv",
            "recipes": "recipes.csv",
            "train_reviews": "train_reviews.csv",
            "test_reviews": "test_reviews.csv"
        }
        self.paths = {k: os.path.join(self.data_folder, v) for k, v in self.files.items()}

    def load_reviews(self):
        return pd.read_csv(self.paths["reviews"])

    def load_recipes(self):
        return pd.read_csv(self.paths["recipes"])

    def load_train_reviews(self):
        return pd.read_csv(self.paths["train_reviews"])

    def load_test_reviews(self):
        return pd.read_csv(self.paths["test_reviews"])

    def get_available_files(self):
        return [f for f in self.files.values() if os.path.exists(os.path.join(self.data_folder, f))]